In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

# 初始化并创建spark对象
spark = SparkSession.builder.master('local').appName('user_meal').getOrCreate()

# 从hdfs中读取数据
data = spark.read.format('csv').load('/keshe/user_meal.csv',header=True,encoding='utf-8')
# 读取本地数据，参数header=True为使用第一行作为列的名称
# data = spark.read.format('csv').load("file:///home/hadoop/jupyternotebook/user_meal.csv",header = True,encoding='gbk')

In [2]:
# 创建临时表
data.createOrReplaceTempView('user')

In [3]:
# 分析统计各评分数量
rating_num = spark.sql("SELECT Rating, COUNT(Rating) AS num FROM user GROUP BY Rating ORDER BY Rating")
rating_num.show()

+------+-----+
|Rating|  num|
+------+-----+
|     1| 1718|
|     2| 1885|
|     3| 4475|
|     4| 9142|
|     5|21025|
+------+-----+



In [4]:
# 分析rating和mealID联合起来的每道菜的平均评分

from pyspark.sql.functions import avg, col

# 假设存在一个名为ratings的DataFrame，包含mealID和rating两列
# 按照mealID进行分组，并计算每组的平均评分
avg_rating = spark.sql("SELECT mealID, AVG(rating) AS avg_rating FROM user GROUP BY mealID ORDER BY avg_rating DESC")

# 打印评分前100结果
avg_rating.show(100)

+----------+------------------+
|    mealID|        avg_rating|
+----------+------------------+
|B001I8O2MK|               5.0|
|B003FGNS9Q|               5.0|
|B00627UN0C|               5.0|
|B00I9M44E4|               5.0|
|B00HUKCIAS|               5.0|
|B00AEFSISW|               5.0|
|B00CHU542K|               5.0|
|B00CD58R6Y|               5.0|
|B00FJ04ZWK|               5.0|
|B002T33I64|               5.0|
|B008CS6WJK|               5.0|
|B00BR34W3U|               5.0|
|B00332YHLU|               5.0|
|B0051HIK04|               5.0|
|B00BENM4JC|               5.0|
|B009KZYJ62|               5.0|
|B004BHGDYK|               5.0|
|B00D6MQ6ZM|               5.0|
|B007MS58AA|               5.0|
|B00AYU5ULA|               5.0|
|B00H8XP4W6|               5.0|
|B009PNW4LG|               5.0|
|B00BOLEKKA|               5.0|
|B00A1ZV2KG|               5.0|
|B00APE00ZG|               5.0|
|B005WGBBS2|               5.0|
|B003T0M4U2|               5.0|
|B004E2BY8W|               5.0|
|B008DR2

In [5]:
# 分析统计各菜品累积评分
meal_sum = spark.sql("SELECT mealID, SUM(rating) AS sum_rating FROM user GROUP BY mealID ORDER BY sum_rating DESC")
meal_sum.show(20)

+----------+----------+
|    mealID|sum_rating|
+----------+----------+
|B00I3MPDP4|    1973.0|
|B00APE00H4|    1858.0|
|B00DAHSVYC|    1760.0|
|B00I3MMN4I|    1541.0|
|B00CDBTQCW|    1494.0|
|B00B8P8O9K|    1423.0|
|B00I3MNGCG|    1401.0|
|B009FZFONO|    1308.0|
|B006Z48TZS|    1275.0|
|B00I3MNVBW|    1245.0|
|B004MWZLYC|    1178.0|
|B00CDBR1P6|    1133.0|
|B005544TRQ|    1108.0|
|B008QTTGGG|    1068.0|
|B00F87ZUYG|    1028.0|
|B00D5OZQUC|     992.0|
|B00I3MMTS8|     980.0|
|B00DTOYIIE|     976.0|
|B00DQISQX6|     893.0|
|B008BQ8VOC|     890.0|
+----------+----------+
only showing top 20 rows



In [6]:
# 词频统计
Review_sum = spark.sql("SELECT Review, COUNT(*) AS num FROM user GROUP BY Review ORDER BY num DESC")
Review_sum.show(10)

+--------------------+-----+
|              Review|  num|
+--------------------+-----+
|太美味了，强烈推荐！|12382|
|          简直太赞了| 6234|
|        非常非常好吃| 3752|
|    很美味，推荐品尝| 3721|
|    此味只应天上有！| 2955|
|            味道很正| 1911|
|        有特色，卫生| 1841|
|  尝过之后，不得不赞| 1257|
|        有特色，好吃|  835|
|              基本OK|  786|
+--------------------+-----+
only showing top 10 rows



In [7]:
# 使用Spark SQL进行数据查询和处理每道菜出现最多的词
result = spark.sql("SELECT mealID, Review, COUNT(*) as count FROM user GROUP BY mealID, Review ORDER BY count DESC")

# 输出结果
result.show()

+----------+--------------------+-----+
|    mealID|              Review|count|
+----------+--------------------+-----+
|B00APE00H4|太美味了，强烈推荐！|  207|
|B009FZFONO|太美味了，强烈推荐！|  150|
|B00I3MPDP4|太美味了，强烈推荐！|  129|
|B00B8P8O9K|太美味了，强烈推荐！|  118|
|B006Z48TZS|太美味了，强烈推荐！|  117|
|B004MWZLYC|太美味了，强烈推荐！|  112|
|B00DTOYIIE|太美味了，强烈推荐！|  106|
|B005544TRQ|太美味了，强烈推荐！|  105|
|B00F87ZUYG|太美味了，强烈推荐！|  103|
|B00DAHSVYC|太美味了，强烈推荐！|   93|
|B008QTTGGG|太美味了，强烈推荐！|   91|
|B00I3MNGCG|太美味了，强烈推荐！|   89|
|B0099JKR6U|太美味了，强烈推荐！|   88|
|B00H7NDSPC|太美味了，强烈推荐！|   86|
|B008BQ8VOC|太美味了，强烈推荐！|   85|
|B00I3MPDP4|          简直太赞了|   84|
|B00CDBTQCW|太美味了，强烈推荐！|   82|
|B00DQISQX6|太美味了，强烈推荐！|   80|
|B00F0XPJH6|太美味了，强烈推荐！|   75|
|B00D5OZQUC|太美味了，强烈推荐！|   73|
+----------+--------------------+-----+
only showing top 20 rows



In [8]:
from pyspark.sql.functions import col
from pyecharts import options as opts
from pyecharts.charts import Pie

# 加载数据并创建DataFrame
review = spark.read.csv("file:///home/hadoop/jupyternotebook/keshe/user_meal.csv", header=True, inferSchema=True,encoding='utf-8')

# 统计评分数量
score_count = review.groupBy("Rating").count().collect()
score_dict = {str(x[0]): int(x[1]) for x in score_count}

# 对评分数量进行排序
sorted_score = sorted(score_dict.items(), key=lambda x: x[1], reverse=False)

# 绘制饼图
pie = (
    Pie()
    .add("", [(k, v) for k, v in sorted_score])
    .set_colors(["#FFE4E1",  "#D8BFD8","#F5DEB3", "#F0E68C", "#ADD8E6"])
    .set_global_opts(title_opts=opts.TitleOpts(title="评分数量"), 
                     legend_opts=opts.LegendOpts(pos_right="15%"))
    .set_series_opts(label_opts=opts.LabelOpts(formatter="评分：{b},  数量: {c}, 百分比：({d}%)"))
)

# 输出饼图
pie.render("/home/hadoop/jupyternotebook/keshe/tu/review_rating_pie.html")

'/home/hadoop/jupyternotebook/keshe/tu/review_rating_pie.html'

In [19]:
from pyspark.sql.functions import avg
from pyspark.sql.functions import count
from pyecharts.charts import Bar

# 按照菜品ID进行分组并计算平均评分
meal_ratings = review.groupBy("MealID").agg(
    avg("Rating").alias("avg_rating"),
    count("ReviewTime").alias("review_count")
)

# 按照平均评分进行升序排序并选择前20个菜品
bottom_ratings = meal_ratings.orderBy("avg_rating").limit(20)

# 创建柱状图实例
bar = (
    Bar(init_opts=opts.InitOpts(width="1200px", height="600px"))
    .add_xaxis([row["MealID"] for row in bottom_ratings.collect()])
    .add_yaxis("平均评分", [row["avg_rating"] for row in bottom_ratings.collect()])
    .set_global_opts(
        title_opts=opts.TitleOpts(title="平均评分最低的20个菜品"),
        xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=-45),name="菜品ID"),
        yaxis_opts=opts.AxisOpts(name="平均评分"),
        toolbox_opts=opts.ToolboxOpts()
    )
)

# 输出柱状图
bar.render("/home/hadoop/jupyternotebook/keshe/tu/avg_rating_bottom20_meals.html")

'/home/hadoop/jupyternotebook/keshe/tu/avg_rating_bottom20_meals.html'

In [10]:
from pyecharts.globals import ThemeType
from pyecharts.charts import Funnel
# 将评价数量的统计表转换
review_pd = review.toPandas()
# 分组统计评价数量
Review_sum1 = review_pd.groupby(['Review']).size() \
.reset_index(name='num').sort_values('num', ascending=False)
# 生成漏斗图
funnel = Funnel(
    init_opts=opts.InitOpts(theme=ThemeType.LIGHT,width="1600px", height="700px")
)
funnel.add("评价数量统计图", list(zip(Review_sum1['Review'], Review_sum1['num'])))
funnel.render('/home/hadoop/jupyternotebook/keshe/tu/loudou_review.html')

'/home/hadoop/jupyternotebook/keshe/tu/loudou_review.html'

In [11]:
import jieba
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyecharts import options as opts
from pyecharts.charts import WordCloud

# 对评价进行分词
cut_text_udf = udf(lambda x: " ".join(jieba.cut(x)), StringType())
review_cut = review.select("Review").withColumn("cut_text", cut_text_udf("Review"))

# 统计各个词的出现次数
word_count = review_cut.rdd.flatMap(lambda x: x[1].split()).map(lambda x: (x, 1)).reduceByKey(lambda a, b: a + b).collect()

# 绘制词云图
wordcloud = (
    WordCloud()
    .add("", word_count, word_size_range=[20, 100])
    .set_global_opts(title_opts=opts.TitleOpts(title="评价词云图"))
)

# 输出词云图
wordcloud.render("/home/hadoop/jupyternotebook/keshe/tu/review_wordcloud.html")

'/home/hadoop/jupyternotebook/keshe/tu/review_wordcloud.html'

In [12]:
import random
province_dict = {
    1:'北京省',2:'天津省',3:'河北省',4:'山西省',5:'内蒙古省',6:'辽宁省',7:'吉林省',8:'黑龙江省',
    9:'上海省',10:'江苏省',11:'浙江省',12:'安徽省',13:'福建省',14:'江西省',15:'山东省',16:'河南省',
    17:'湖北省',18:'湖南省',19:'广东省',20:'广西省',21:'海南省',22:'四川省',23:'贵州省',
    24:'云南省',25:'广西省',26:'西藏省',27:'陕西省',28:'甘肃省',29:'青海省',30:'宁夏省',
    31:'新疆省',32:'台湾省'
   
}
province = []
for x in range(len(review_pd)):
    province.append(province_dict[random.randint(1,32)])
review_pd['province'] = province
review_pd.head(10)

,UserID,Rating,ReviewTime,Review,MealID,province
0,A2A6NH6DPE0VXR,4,1493576000,非常非常好吃,B000W4WD40,新疆省
1,A1MNDBR7DF0EU9,5,1493576000,太美味了，强烈推荐！,B002BLCNHY,江苏省
2,AT1BYQVGK7U71,5,1493576000,简直太赞了,B001SE07JG,广西省
3,A328S9RN3U5M68,3,1493576000,味道很正,B001PN63PC,四川省
4,A16H208JVRTMU4,5,1493576000,太美味了，强烈推荐！,B000WT7R6O,浙江省
5,ATDNMB4EB7ZY4,2,1493576000,有特色，好吃,B000NHRTAO,吉林省
6,A3VNYHAEKTHVPY,3,1493577600,非常非常好吃,B005GT575S,山东省
7,A13MM7UES60AAU,5,1493592000,太美味了，强烈推荐！,B008X0SGDC,江苏省
8,A3TNYNA2360NPA,5,1493592000,太美味了，强烈推荐！,B008QTTGGG,新疆省
9,A206S2JFUZ5WT1,5,1493592000,太美味了，强烈推荐！,B008O2QERY,湖北省


In [13]:
# 绘制地图
review_pd1 = review_pd.groupby(['province']).size() \
.reset_index(name='Rating').sort_values('Rating', ascending=False)

from pyecharts.charts import Map
user_rating_province = Map()
user_rating_province.add("", list(zip(review_pd1['province'],review_pd1['Rating'])), "china")
user_rating_province.set_global_opts(
    title_opts=opts.TitleOpts(title="用户评分地域分布图"),
    visualmap_opts=opts.VisualMapOpts(min_=0, max_=1500, is_piecewise=True),
)
user_rating_province.render("/home/hadoop/jupyternotebook/keshe/tu/user_rating_province.html")

'/home/hadoop/jupyternotebook/keshe/tu/user_rating_province.html'

In [33]:
import pymysql
# 将可视化结果存入mysql数据库
paths = [
    (1,'/home/hadoop/jupyternotebook/keshe/tu/review_rating_pie.html'),
    (2,'/home/hadoop/jupyternotebook/keshe/tu/avg_rating_bottom20_meals.html'), 
    (3,'/home/hadoop/jupyternotebook/keshe/tu/loudou_review.html'),
    (4,'/home/hadoop/jupyternotebook/keshe/tu/user_rating_province.html')
]
insert_sql = "INSERT INTO path(path_id,url) VALUES (%s, %s)"
# 创建MySQL数据库连接
conn = pymysql.connect(
    host="localhost",
    user="root",
    passwd="fsd009750",
    port=3306,
    db="spark",
    charset="utf8mb4"
)
#print(conn)
# 创建游标
cursor = conn.cursor()
#执行插入多条记录的操作
cursor.executemany(insert_sql, paths)
conn.commit()
# 关闭数据库连接
conn.close()

In [ ]:
#spark.stop()